## Imports

In [1]:
# Importaciones necesarias
import requests
import pandas as pd
import os
import json
import time
import boto3
from datetime import datetime, timedelta
from dotenv import load_dotenv

## Local

In [7]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()
AEMET_API_KEY = os.getenv("AEMET_API_KEY")
AEMET_BASE_URL = "https://opendata.aemet.es/opendata/api"

# Directorio local para guardar los archivos
OUTPUT_BASE_DIRECTORY = "data/raw/aemet-diaria-2025"
os.makedirs(OUTPUT_BASE_DIRECTORY, exist_ok=True)

def extractorAEMET_Diario(fecha_inicio: str, fecha_fin: str):
    if not AEMET_API_KEY:
        print("Error: AEMET_API_KEY no encontrada. Por favor, asegúrate de que esté configurada.")
        return []
   
    fecha_ini_api_format = f"{fecha_inicio}T00:00:00UTC"
    fecha_fin_api_format = f"{fecha_fin}T23:59:59UTC"
    endpoint_url = (
        f"{AEMET_BASE_URL}/valores/climatologicos/diarios/datos/"
        f"fechaini/{fecha_ini_api_format}/fechafin/{fecha_fin_api_format}/todasestaciones"
    )
    print(f"  Solicitando datos para el período: {fecha_inicio} a {fecha_fin}")
   
    try:
        response_inicial = requests.get(
            endpoint_url,
            params={'api_key': AEMET_API_KEY},
            headers={'Cache-Control': 'no-cache'},
            timeout=30
        )
        response_inicial.raise_for_status()
        data_inicial = response_inicial.json()
        if response_inicial.status_code != 200 or 'datos' not in data_inicial:
            print(f"  Error en la solicitud inicial o no hay datos disponibles. Código: {response_inicial.status_code}. Detalles: {data_inicial.get('descripcion', 'N/A')}")
            return []
        datos_url = data_inicial.get('datos')
        if not datos_url:
            print(f"  Error: No se encontró la URL de los datos reales en la respuesta inicial para {fecha_inicio}-{fecha_fin}.")
            print(f"  Respuesta inicial de AEMET: {data_inicial}")
            return []
        time.sleep(1.0)
        response_datos = requests.get(datos_url)
        response_datos.raise_for_status()
        batch_data = response_datos.json()

        if isinstance(batch_data, list) and batch_data:
            print(f"  Datos recibidos para el lote: {len(batch_data)} registros.")

            batch_data = batch_data[:10000]  # Limitar a 10,000 registros
            
            # Guardar los datos en un archivo JSON
            output_filename = f"{fecha_inicio}_{fecha_fin}.json"
            local_file_path = os.path.join(OUTPUT_BASE_DIRECTORY, output_filename)
            with open(local_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(batch_data, json_file, indent=4, ensure_ascii=False)
            
            print(f"Archivo generado: {local_file_path}")
            return batch_data
        else:
            print(f"  La respuesta de datos para {fecha_inicio}-{fecha_fin} no contiene una lista o está vacía.")
            return []
    except requests.exceptions.RequestException as e:
        print(f"  Error de conexión o HTTP para el período {fecha_inicio}-{fecha_fin}: {e}")
        return []
    except ValueError as e:
        print(f"  Error al parsear la respuesta JSON para el período {fecha_inicio}-{fecha_fin}: {e}")
        return []
    except Exception as e:
        print(f"  Ocurrió un error inesperado durante la extracción del lote: {e}")
        return []

def generar_archivos_por_rango():
    start_date = datetime(2025, 1, 1)
    end_date = datetime.now() - timedelta(days=7)  # Ajustar para una semana antes del día actual
    current_start = start_date

    while current_start < end_date:
        current_end = current_start + timedelta(days=14)
        if current_end > end_date:
            current_end = end_date

        fecha_inicio = current_start.strftime("%Y-%m-%d")
        fecha_fin = current_end.strftime("%Y-%m-%d")
        print(f"\nProcesando rango: {fecha_inicio} a {fecha_fin}")
        datos = extractorAEMET_Diario(fecha_inicio, fecha_fin)

        current_start = current_end + timedelta(days=1)

if __name__ == "__main__":
    generar_archivos_por_rango()


Procesando rango: 2025-01-01 a 2025-01-15
  Solicitando datos para el período: 2025-01-01 a 2025-01-15
  Datos recibidos para el lote: 13226 registros.
Archivo generado: data/raw/aemet-diaria-2025\2025-01-01_2025-01-15.json

Procesando rango: 2025-01-16 a 2025-01-30
  Solicitando datos para el período: 2025-01-16 a 2025-01-30
  Datos recibidos para el lote: 13274 registros.
Archivo generado: data/raw/aemet-diaria-2025\2025-01-16_2025-01-30.json

Procesando rango: 2025-01-31 a 2025-02-14
  Solicitando datos para el período: 2025-01-31 a 2025-02-14
  Datos recibidos para el lote: 13280 registros.
Archivo generado: data/raw/aemet-diaria-2025\2025-01-31_2025-02-14.json

Procesando rango: 2025-02-15 a 2025-03-01
  Solicitando datos para el período: 2025-02-15 a 2025-03-01
  Datos recibidos para el lote: 13272 registros.
Archivo generado: data/raw/aemet-diaria-2025\2025-02-15_2025-03-01.json

Procesando rango: 2025-03-02 a 2025-03-16
  Solicitando datos para el período: 2025-03-02 a 2025-03

## AWS Bucket

In [10]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()
AEMET_API_KEY = os.getenv("AEMET_API_KEY")
AEMET_BASE_URL = "https://opendata.aemet.es/opendata/api"

# Configuración del bucket de S3
S3_BUCKET_NAME = "hab-bucket-prueba"
S3_REGION = "eu-north-1"
S3_OUTPUT_DIRECTORY = "data/raw/aemet-diaria-2025"

def upload_to_s3(file_path, s3_key):
    try:
        s3 = boto3.client("s3", region_name=S3_REGION)
        s3.upload_file(file_path, S3_BUCKET_NAME, s3_key)
        print(f"Archivo subido exitosamente a S3: s3://{S3_BUCKET_NAME}/{s3_key}")
    except Exception as e:
        print(f"Error al subir el archivo a S3: {e}")

def extractorAEMET_Diario(fecha_inicio: str, fecha_fin: str):
    if not AEMET_API_KEY:
        print("Error: AEMET_API_KEY no encontrada. Por favor, asegúrate de que esté configurada.")
        return []
   
    fecha_ini_api_format = f"{fecha_inicio}T00:00:00UTC"
    fecha_fin_api_format = f"{fecha_fin}T23:59:59UTC"
    endpoint_url = (
        f"{AEMET_BASE_URL}/valores/climatologicos/diarios/datos/"
        f"fechaini/{fecha_ini_api_format}/fechafin/{fecha_fin_api_format}/todasestaciones"
    )
    print(f"  Solicitando datos para el período: {fecha_inicio} a {fecha_fin}")
   
    try:
        response_inicial = requests.get(
            endpoint_url,
            params={'api_key': AEMET_API_KEY},
            headers={'Cache-Control': 'no-cache'},
            timeout=60
        )
        response_inicial.raise_for_status()
        data_inicial = response_inicial.json()
        if response_inicial.status_code != 200 or 'datos' not in data_inicial:
            print(f"  Error en la solicitud inicial o no hay datos disponibles. Código: {response_inicial.status_code}. Detalles: {data_inicial.get('descripcion', 'N/A')}")
            return []
        datos_url = data_inicial.get('datos')
        if not datos_url:
            print(f"  Error: No se encontró la URL de los datos reales en la respuesta inicial para {fecha_inicio}-{fecha_fin}.")
            print(f"  Respuesta inicial de AEMET: {data_inicial}")
            return []
        time.sleep(1.0)
        response_datos = requests.get(datos_url)
        response_datos.raise_for_status()
        batch_data = response_datos.json()

        if isinstance(batch_data, list) and batch_data:
            print(f"  Datos recibidos para el lote: {len(batch_data)} registros.")

            batch_data = batch_data[:10000]  # Limitar a 10,000 registros
            
            # Guardar los datos en un archivo JSON
            output_filename = f"{fecha_inicio}_{fecha_fin}.json"
            local_file_path = os.path.join(OUTPUT_BASE_DIRECTORY, output_filename)
            with open(local_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(batch_data, json_file, indent=4, ensure_ascii=False)
            
            print(f"Archivo generado: {local_file_path}")

            # Subir el archivo a S3
            s3_key = f"{S3_OUTPUT_DIRECTORY}/{output_filename}"
            upload_to_s3(local_file_path, s3_key)

            return batch_data
        else:
            print(f"  La respuesta de datos para {fecha_inicio}-{fecha_fin} no contiene una lista o está vacía.")
            return []
    except requests.exceptions.RequestException as e:
        print(f"  Error de conexión o HTTP para el período {fecha_inicio}-{fecha_fin}: {e}")
        return []
    except ValueError as e:
        print(f"  Error al parsear la respuesta JSON para el período {fecha_inicio}-{fecha_fin}: {e}")
        return []
    except Exception as e:
        print(f"  Ocurrió un error inesperado durante la extracción del lote: {e}")
        return []

def generar_archivos_por_rango():
    start_date = datetime(2025, 1, 1)
    end_date = datetime.now() - timedelta(days=7)  # Ajustar para una semana antes del día actual
    current_start = start_date

    while current_start < end_date:
        current_end = current_start + timedelta(days=14)
        if current_end > end_date:
            current_end = end_date

        fecha_inicio = current_start.strftime("%Y-%m-%d")
        fecha_fin = current_end.strftime("%Y-%m-%d")
        print(f"\nProcesando rango: {fecha_inicio} a {fecha_fin}")
        datos = extractorAEMET_Diario(fecha_inicio, fecha_fin)

        current_start = current_end + timedelta(days=1)

if __name__ == "__main__":
    generar_archivos_por_rango()


Procesando rango: 2025-01-01 a 2025-01-15
  Solicitando datos para el período: 2025-01-01 a 2025-01-15
  Datos recibidos para el lote: 13226 registros.
Archivo generado: data/raw/aemet-diaria-2025\2025-01-01_2025-01-15.json
Error al subir el archivo a S3: Failed to upload data/raw/aemet-diaria-2025\2025-01-01_2025-01-15.json to hab-bucket-prueba/data/raw/aemet-diaria-2025/2025-01-01_2025-01-15.json: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

Procesando rango: 2025-01-16 a 2025-01-30
  Solicitando datos para el período: 2025-01-16 a 2025-01-30
  Datos recibidos para el lote: 13274 registros.
Archivo generado: data/raw/aemet-diaria-2025\2025-01-16_2025-01-30.json
Error al subir el archivo a S3: Failed to upload data/raw/aemet-diaria-2025\2025-01-16_2025-01-30.json to hab-bucket-prueba/data/raw/aemet-diaria-2025/2025-01-16_2025-01-30.json: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

Procesando rango: 202